Breaking down the files into the YOLO format and structure:

Folder structure:
* datasets
     * train
          * images
          * labels
     * test
          * images
          * labels
     * val
          * images
          * labels
     * (data.yaml)

label files target format:  
| detected class | bbox X-position | bbox Y-position | bbox Width | bbox Height |
|-|-|-|-|-|
| int | float (0<=X<1) | float (0<=Y<1) | float (0<=W<1) | float (0<=H<1) |


In [1]:
import json
import os

In [2]:
with open('PKLot/train/_annotations.coco.json', 'r') as file:
    tr_annotations = json.load(file)

In [3]:
tr_annotations.keys()

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])

In [4]:
tr_annotations['categories']

[{'id': 0, 'name': 'spaces', 'supercategory': 'none'},
 {'id': 1, 'name': 'space-empty', 'supercategory': 'spaces'},
 {'id': 2, 'name': 'space-occupied', 'supercategory': 'spaces'}]

In [5]:
tr_annotations['images'][0]

{'id': 0,
 'license': 1,
 'file_name': '2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d8997ce513fd8d124.jpg',
 'height': 640,
 'width': 640,
 'date_captured': '2021-01-05T21:55:11+00:00'}

In [6]:
tr_annotations['annotations'][0]

{'id': 0,
 'image_id': 0,
 'category_id': 2,
 'bbox': [304, 468, 83.5, 114],
 'area': 9519,
 'segmentation': [],
 'iscrowd': 0}

id, filename, height, width, bbox will be used to create the label files.
* id for merging in pandas
* filename.jpg -> filename.txt
* bbox [x,y,w,h] -> [X,Y,W,H]
* height, width -> float calculation

In [7]:
import pandas as pd
df_left = pd.DataFrame(tr_annotations['images'])
df_left

,id,license,file_name,height,width,date_captured
0,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00
1,1,1,2012-10-28_15_07_11_jpg.rf.bf0215bf1be731828c9...,640,640,2021-01-05T21:55:11+00:00
2,2,1,2012-10-15_15_16_04_jpg.rf.bf069ee1dcc96c9e5c2...,640,640,2021-01-05T21:55:11+00:00
3,3,1,2013-03-20_11_50_06_jpg.rf.bf0daeaae52ff23b8cc...,640,640,2021-01-05T21:55:11+00:00
4,4,1,2012-12-24_09_50_05_jpg.rf.bf1406d0cd01a8d266d...,640,640,2021-01-05T21:55:11+00:00
...,...,...,...,...,...,...
8686,8686,1,2012-12-14_15_55_12_jpg.rf.36f11c96000ddbc285b...,640,640,2021-01-05T21:55:11+00:00
8687,8687,1,2012-10-23_09_00_52_jpg.rf.36f853be166800e0af6...,640,640,2021-01-05T21:55:11+00:00
8688,8688,1,2013-03-03_11_45_06_jpg.rf.36fb00d7e5aa7d56587...,640,640,2021-01-05T21:55:11+00:00
8689,8689,1,2013-03-19_16_50_12_jpg.rf.37070902b62b1b5f5fe...,640,640,2021-01-05T21:55:11+00:00


In [8]:
df_right = pd.DataFrame(tr_annotations['annotations'])
df_right

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,2,"[304, 468, 83.5, 114]",9519.00,[],0
1,1,0,2,"[271, 390, 76.5, 89]",6808.50,[],0
2,2,0,2,"[245, 326, 67.5, 80]",5400.00,[],0
3,3,0,2,"[223, 270, 63.5, 73]",4635.50,[],0
4,4,0,2,"[204, 224, 58, 57]",3306.00,[],0
...,...,...,...,...,...,...,...
497851,497851,8690,2,"[389, 38, 39.5, 36.5]",1441.75,[],0
497852,497852,8690,2,"[366, 11, 44.5, 36.5]",1624.25,[],0
497853,497853,8690,2,"[561, 75, 29.5, 61.5]",1814.25,[],0
497854,497854,8690,2,"[529, 50, 30.5, 50.5]",1540.25,[],0


In [9]:
df = pd.merge(df_left, df_right, how='left', left_on='id', right_on='image_id')
df

,id_x,license,file_name,height,width,date_captured,id_y,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,0.0,0.0,2.0,"[304, 468, 83.5, 114]",9519.00,[],0.0
1,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,1.0,0.0,2.0,"[271, 390, 76.5, 89]",6808.50,[],0.0
2,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,2.0,0.0,2.0,"[245, 326, 67.5, 80]",5400.00,[],0.0
3,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,3.0,0.0,2.0,"[223, 270, 63.5, 73]",4635.50,[],0.0
4,0,1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2021-01-05T21:55:11+00:00,4.0,0.0,2.0,"[204, 224, 58, 57]",3306.00,[],0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
498040,8690,1,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,2021-01-05T21:55:11+00:00,497851.0,8690.0,2.0,"[389, 38, 39.5, 36.5]",1441.75,[],0.0
498041,8690,1,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,2021-01-05T21:55:11+00:00,497852.0,8690.0,2.0,"[366, 11, 44.5, 36.5]",1624.25,[],0.0
498042,8690,1,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,2021-01-05T21:55:11+00:00,497853.0,8690.0,2.0,"[561, 75, 29.5, 61.5]",1814.25,[],0.0
498043,8690,1,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,2021-01-05T21:55:11+00:00,497854.0,8690.0,2.0,"[529, 50, 30.5, 50.5]",1540.25,[],0.0


In [10]:
df = df.drop([
    'id_x', 
    'id_y',
    'license', 
    'date_captured', 
    'image_id', 
    'area', 
    'segmentation', 
    'iscrowd'], 
    axis=1)


In [11]:
df.isna().value_counts()

file_name  height  width  category_id  bbox 
False      False   False  False        False    497856
                          True         True        189
Name: count, dtype: int64

In [12]:
df = df.dropna()
df.isna().value_counts()

file_name  height  width  category_id  bbox 
False      False   False  False        False    497856
Name: count, dtype: int64

In [13]:
df

,file_name,height,width,category_id,bbox
0,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2.0,"[304, 468, 83.5, 114]"
1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2.0,"[271, 390, 76.5, 89]"
2,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2.0,"[245, 326, 67.5, 80]"
3,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2.0,"[223, 270, 63.5, 73]"
4,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,2.0,"[204, 224, 58, 57]"
...,...,...,...,...,...
498040,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,2.0,"[389, 38, 39.5, 36.5]"
498041,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,2.0,"[366, 11, 44.5, 36.5]"
498042,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,2.0,"[561, 75, 29.5, 61.5]"
498043,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,2.0,"[529, 50, 30.5, 50.5]"


In [14]:
# Change category numbers to start from 0
print(df['category_id'].value_counts())
df['category_id'].loc[df['category_id'] == 1] = 0
df['category_id'].loc[df['category_id'] == 2] = 1
print(df['category_id'].value_counts())
df

category_id
1.0    265908
2.0    231948
Name: count, dtype: int64
category_id
0.0    265908
1.0    231948
Name: count, dtype: int64


/tmp/ipykernel_478332/1974903249.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['category_id'].loc[df['category_id'] == 1] = 0
/tmp/ipykernel_478332/1974903249.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pan

,file_name,height,width,category_id,bbox
0,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1.0,"[304, 468, 83.5, 114]"
1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1.0,"[271, 390, 76.5, 89]"
2,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1.0,"[245, 326, 67.5, 80]"
3,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1.0,"[223, 270, 63.5, 73]"
4,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1.0,"[204, 224, 58, 57]"
...,...,...,...,...,...
498040,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,1.0,"[389, 38, 39.5, 36.5]"
498041,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,1.0,"[366, 11, 44.5, 36.5]"
498042,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,1.0,"[561, 75, 29.5, 61.5]"
498043,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,1.0,"[529, 50, 30.5, 50.5]"


In [15]:
df['category_id'] = df['category_id'].convert_dtypes()
df

,file_name,height,width,category_id,bbox
0,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1,"[304, 468, 83.5, 114]"
1,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1,"[271, 390, 76.5, 89]"
2,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1,"[245, 326, 67.5, 80]"
3,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1,"[223, 270, 63.5, 73]"
4,2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d899...,640,640,1,"[204, 224, 58, 57]"
...,...,...,...,...,...
498040,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,1,"[389, 38, 39.5, 36.5]"
498041,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,1,"[366, 11, 44.5, 36.5]"
498042,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,1,"[561, 75, 29.5, 61.5]"
498043,2013-04-09_15_55_11_jpg.rf.3709f85e4b3525701db...,640,640,1,"[529, 50, 30.5, 50.5]"


In [16]:
print(df.loc[1, 'file_name'])
df['file_name'] = df['file_name'].apply(func=lambda x: x[:len(x)-4]+'.txt' if str.endswith(x, '.jpg') else x)
print(df.loc[1, 'file_name'])

2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d8997ce513fd8d124.jpg
2013-04-10_12_00_06_jpg.rf.bef78f7879666a6d8997ce513fd8d124.txt


In [17]:
labels_path = 'datasets/train/labels'

for row, cols in df.iterrows():
    x,y,w,h = cols['bbox']
    X = (x+w / 2) / cols['width']
    Y = (y+h / 2) / cols['height']
    W = w / cols['width']
    H = h / cols['height']
    line = str(cols['category_id']) + ' ' + str(X) + ' ' + str(Y) + ' ' + str(W) + ' ' + str(H)+'\n'
    label_path = os.path.join(labels_path, cols['file_name'])
    with open(label_path, 'a') as file:
        file.writelines(line)